In [1]:
import cv2
import os
#videos 路徑
videoPath=r'videos'
#新videos存放路徑
newVideoPath=r'videos\combine'
#Video檔案 類型
videoType='.avi'
#貨箱編號路徑
numberPath=r'videos\number'
#貨箱編號檔案 類型
numberType='.txt'
#Label路徑
labelPath=r'runs\detect\exp\labels'
#Label檔案 類型
labelType='.txt'
videoFiles=os.listdir(videoPath)
for video in videoFiles:
    videoName=video.replace(videoType,"")
    if os.path.isfile(os.path.join(videoPath,videoName+videoType)):
        #貨箱編號
        number=''
        if os.path.isfile(os.path.join(numberPath,videoName+numberType)):
            nTxt= open(os.path.join(numberPath,videoName+numberType), 'r')
            number=nTxt.read()
            nTxt.close()
        #讀取影片
        cap = cv2.VideoCapture(os.path.join(videoPath,videoName+videoType))
        #影片幀數
        fps_video = cap.get(cv2.CAP_PROP_FPS)
        #影片格式
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        #寬度
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        #高度
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        videoWriter = cv2.VideoWriter(os.path.join(newVideoPath,videoName+videoType), fourcc, fps_video, (frame_width, frame_height))
        frame_id = 0
        while (cap.isOpened()):
            ret, frame = cap.read()
            if ret == True:
                frame_id += 1
                labelName=videoName+"_"+str(frame_id)
                label=os.path.join(labelPath,labelName+labelType)
                if os.path.isfile(label):
                    txt= open(label, 'r')
                    cW=0
                    cH=0
                    rW=0
                    rH=0
                    for infos in txt.readlines():
                        info=infos.split()
                        if rH<float(info[4]):
                            cW=float(info[1])
                            cH=float(info[2])
                            rW=float(info[3])
                            rH=float(info[4])
                    txt.close()
                    left_x_up = int((frame_width*cW)-(frame_width*rW/2))
                    left_y_up = int((frame_height*cH)-(frame_height*rH/2))
                    right_x_down = int((frame_width*cW)+(frame_width*rW/2))
                    right_y_down = int((frame_height*cH)+(frame_height*rH/2))
                    #文字坐标
                    word_x = left_x_up
                    word_y = right_y_down+30
                    cv2.rectangle(frame, (left_x_up, left_y_up), (right_x_down, right_y_down), (55,255,155), 5)
                    cv2.putText(frame,number, (word_x, word_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,255,155), 2)
                videoWriter.write(frame)
            else:
                videoWriter.release()
                break